<a href="https://colab.research.google.com/github/VivekanandaMudelli/hand_drawn_sketch_recognition/blob/main/KNN(Pca).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gdown
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
file_id = "1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE"

# Construct the direct download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
output = "data.csv"  # Change filename as needed
gdown.download(download_url, output, quiet=False)

# Read the CSV file
data = pd.read_csv(output)
data = data.dropna()

X = data.drop(columns = ['label'])
y = data['label']

print(X.shape)
print(y.shape)

Downloading...
From (original): https://drive.google.com/uc?id=1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE
From (redirected): https://drive.google.com/uc?id=1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE&confirm=t&uuid=10e5a8f3-18fe-420c-a376-a4d0a0527e59
To: /content/data.csv
100%|██████████| 943M/943M [00:13<00:00, 68.3MB/s]


(20180, 8100)
(20180,)


In [3]:
# Mean
X_mean = X.mean()

# Standard deviation
X_std = X.std()

# Standardization
Z = (X - X_mean) / X_std.replace(0,1e-10)

In [4]:
# Importing PCA
from sklearn.decomposition import PCA

# Taking top 50 dimensions
pca = PCA(n_components=50)
pca.fit(Z)
x_pca = pca.transform(Z)

# Create the dataframe
df = pd.DataFrame(x_pca,
                       columns=['PC{}'.
                       format(i+1)
                        for i in range(50)])
print(df)

             PC1        PC2        PC3        PC4        PC5        PC6  \
0     -13.473498  14.170485  -8.592970  -0.276516  -4.310718 -12.837158   
1      -7.392499  21.470417 -16.521426  -9.054979  10.826065   2.505616   
2     -11.364221   4.819343  -5.701317  -9.767070  -3.730637 -13.074397   
3      -9.464440   4.396213 -14.905356  -3.827592  -1.930113  -7.333161   
4      -3.514694   4.805839 -10.267749   6.538963  -6.552486   2.683322   
...          ...        ...        ...        ...        ...        ...   
20175  -3.464935   7.745460 -29.708590 -10.414636  14.356331   2.594452   
20176  -6.965887   8.512735  -5.489306   1.021764   0.134603  -6.489945   
20177 -10.654562  -4.086706  -9.890633  -5.559357   9.865364   8.734848   
20178   6.125788  -1.236722 -12.704954 -12.068881   0.344311  -0.449105   
20179  -3.484752  13.398968 -10.478203  -7.359180   4.313199  19.720034   

             PC7       PC8        PC9      PC10  ...      PC41      PC42  \
0       0.117528 -2.761

In [5]:
df['label'] = y

In [6]:
def distance(x,y):
  return np.linalg.norm(x-y)

In [7]:
def k_nearest_neighbors(p, points, k):
    dist = [distance(p,p1) for p1 in points]
    neighborId = np.argsort(dist)
    neighbor = [points[i] for i in neighborId[:k]]

    Knn = [neighbor,neighborId[:k]]
    return Knn

In [8]:
import statistics as st

def knn_predict(X_train, y_train, testPoint, k):
  a,b = k_nearest_neighbors(testPoint,X_train,k)# a maps to the list of points and b maps to the list of indices
  k_labels = [y_train[i] for i in b] #list of labels for the k nearest neighbors
  return st.mode(k_labels)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

X_new = df.drop('label', axis=1)
y_new = df['label']

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_new, y_new, train_size=0.7, random_state=42)
X_train=X_train.values
y_train=y_train.values
X_val=X_val.values
y_val=y_val.values
k=13
predictions=[]

In [11]:
for testPoint in tqdm(X_val, desc="Predicting", unit="sample"):
    predictions.append(knn_predict(X_train, y_train, testPoint, k))

Predicting: 100%|██████████| 6054/6054 [06:11<00:00, 16.29sample/s]


In [12]:
accuracy = accuracy_score(y_val, predictions)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Validation Accuracy: 30.13%
